Save inter-taxa relationships into `.pkl` dict for easier use

In [9]:
import pickle
import glob
import networkx as nx 
import matplotlib.pyplot as plt

to_classify = ["sample", "domain", "phylum", "class", "order", "family", "genus"]

disease = "crc"
classifier = 'family'

In [10]:
files = glob.glob('results/'+disease+'/'+disease+'_dicts/*')
taxa_dicts = []

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
        taxa_dicts.append(taxa_dict)
        
# calculate thresholds

# find average value of each
avg_d = {}
total_organisms = {}
num_files = len(files)
num_files_occurred = {}

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
    
    # count total organisms per patient
    organism_count = 0
    for i in taxa_dict[classifier]:
        organism_count += taxa_dict[classifier][i]
        if i not in num_files_occurred:
            num_files_occurred[i] = 1
        else:
            num_files_occurred[i] += 1
    total_organisms[file] = organism_count

    # add normalized data to the total, and divide by number of patients to average
    for i in taxa_dict[classifier]:
        if i not in avg_d:
            avg_d[i] = taxa_dict[classifier][i] / (num_files * organism_count)
        else:
            avg_d[i] += taxa_dict[classifier][i] / (num_files * organism_count)

total_files_per_sample = {}

for file in files:
    with open(file, 'rb') as f:
        taxa_dict = pickle.load(f)
    for i in taxa_dict[classifier]:
        if taxa_dict[classifier][i] / total_organisms[file] < avg_d[i]:
            continue
        else:
            if i not in total_files_per_sample:
                total_files_per_sample[i] = {}
            for j in taxa_dict[classifier]:
                if taxa_dict[classifier][j] / total_organisms[file] >= avg_d[j]:
                    if j not in total_files_per_sample[i]:
                        total_files_per_sample[i][j] = 1
                    else:
                        total_files_per_sample[i][j] += 1   

filepath = "results/file_dicts/"+disease+"_"+classifier+".pkl"

with open(filepath, 'wb') as f:
    pickle.dump(total_files_per_sample, f)

In [6]:
with open(filepath, 'rb') as f:
    total_files_per_sample = pickle.load(f)